In [32]:

body = client_9eac131ca2e04a739138b3d5f5701a44.get_object(Bucket='module8-donotdelete-pr-zozbishnggzy75',Key='coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

neighborhoods = pd.read_csv(body)
neighborhoods.head()


,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values





In [16]:
!pip install folium

     |████████████████████████████████| 94 kB 5.4 MB/s  eta 0:00:01


In [17]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [99]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


This is only Borough which contains Toronto

In [100]:
westtoronto_data = neighborhoods[neighborhoods['Borough'] == 'West Toronto'].reset_index(drop=True)
westtoronto_data.head(10)

,Borough,Neighbourhood,Latitude,Longitude
0,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
1,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
3,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


Creating a map of West Toronto using Latitude and Logitude values

In [46]:

import folium 
#latitude = '43.6606'
#longitude = '79.4692'

map_westtoronto = folium.Map(location=[latitude, longitude], zoom_start=3)

# add markers to map
for lat, lng, label in zip(westtoronto_data['Latitude'], westtoronto_data['Longitude'], westtoronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_westtoronto)  
    
map_westtoronto

In [49]:
CLIENT_ID = '3SHIHVNX2RDNHOZAE0IYPHBGIGGUNMYJSXCM20VTPY4OWIZ2' # your Foursquare ID
CLIENT_SECRET = 'FZG13JSC2IN5XHQROCJQTMBSPH3PIKNGC5JCZP3QAFGRAXUI' # your Foursquare Secret
ACCESS_TOKEN = 'NVAO2NQCRYPOJXSOHEHVLFIGMRHHJ4RETIKQLBNGZ40FVGC3' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value



Exploring the first Neighbourhood name

In [101]:
westtoronto_data.loc[0, 'Neighbourhood']

'Dufferin, Dovercourt Village'

Get the neighborhood's latitude and longitude values.

In [102]:
neighborhood_latitude = westtoronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = westtoronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = westtoronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Dufferin, Dovercourt Village are 43.66900510000001, -79.4422593.


Now, let's get the top 100 venues that are in Duffering, Dovercourt within a radius of 500 meters.

In [103]:
LIMIT = 100 # limit of number of venues returned by Foursquare API


radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=3SHIHVNX2RDNHOZAE0IYPHBGIGGUNMYJSXCM20VTPY4OWIZ2&client_secret=FZG13JSC2IN5XHQROCJQTMBSPH3PIKNGC5JCZP3QAFGRAXUI&v=20180605&ll=43.66900510000001,-79.4422593&radius=500&limit=100'

Send the GET request and examine the resutls

In [104]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600b14620debfa7431648272'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davenport',
  'headerFullLocation': 'Davenport, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 43.67350510450001,
    'lng': -79.43604977526607},
   'sw': {'lat': 43.664505095500004, 'lng': -79.44846882473394}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5753753b498eeb535c53aed5',
       'name': 'The Greater Good Bar',
       'location': {'address': '229 Geary St',
        'crossStreet': 'at Dufferin St',
        'lat': 43.669409,
        'lng': -79.439267,
        'labeledLatLngs': [{'label': 'disp

In [57]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [105]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,FreshCo,Grocery Store,43.667918,-79.440754
3,Blood Brothers Brewing,Brewery,43.669944,-79.436533
4,Happy Bakery & Pastries,Bakery,43.667050,-79.441791


And how many venues were returned by Foursquare?

In [106]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


Explore Neighborhoods in Manhattan and create new venue data frame

In [107]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [108]:
westtoronto_venues = getNearbyVenues(names=westtoronto_data['Neighbourhood'],
                                   latitudes=westtoronto_data['Latitude'],
                                   longitudes=westtoronto_data['Longitude']
                                  )

Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea


Let's check our new dataframe

In [109]:
print(westtoronto_venues.shape)
westtoronto_venues.head()

(161, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dufferin, Dovercourt Village",43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,"Dufferin, Dovercourt Village",43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,"Dufferin, Dovercourt Village",43.669005,-79.442259,FreshCo,43.667918,-79.440754,Grocery Store
3,"Dufferin, Dovercourt Village",43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery
4,"Dufferin, Dovercourt Village",43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery


Let's check how many venues were returned for each neighborhood

In [110]:
westtoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Dufferin, Dovercourt Village",18,18,18,18,18,18
"High Park, The Junction South",25,25,25,25,25,25
"Little Portugal, Trinity",43,43,43,43,43,43
"Parkdale, Roncesvalles",14,14,14,14,14,14
"Runnymede, Swansea",38,38,38,38,38,38



Let's find out how many unique categories can be curated from all the returned venues

In [111]:
print('There are {} uniques categories.'.format(len(westtoronto_venues['Venue Category'].unique())))

There are 81 uniques categories.


Analyze Each Neighborhood

In [112]:
# one hot encoding
westtor_onehot = pd.get_dummies(westtoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
westtor_onehot['Neighborhood'] = westtoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [westtor_onehot.columns[-1]] + list(westtor_onehot.columns[:-1])
westtor_onehot = westtor_onehot[fixed_columns]

westtor_onehot.head()

,Neighborhood,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Burrito Place,Café,Cajun / Creole Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,IT Services,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Liquor Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Portuguese Restaurant,Pub,Record Shop,Restaurant,Sandwich Place,School,Smoothie Shop,Speakeasy,Stadium,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Dufferin, Dovercourt Village",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Dufferin, Dovercourt Village",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Dufferin, Dovercourt Village",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [113]:
westtoronto_grouped = westtor_onehot.groupby('Neighborhood').mean().reset_index()
westtoronto_grouped

,Neighborhood,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Burrito Place,Café,Cajun / Creole Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,IT Services,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Liquor Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Portuguese Restaurant,Pub,Record Shop,Restaurant,Sandwich Place,School,Smoothie Shop,Speakeasy,Stadium,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Brockton, Parkdale Village, Exhibition Place",0.00,0.000000,0.00,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.086957,0.000000,0.043478,0.130435,0.00,0.043478,0.000000,0.086957,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.043478,0.00,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Dufferin, Dovercourt Village",0.00,0.000000,0.00,0.000000,0.111111,0.055556,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.111111,0.055556,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000
2,"High Park, The Junction South",0.04,0.000000,0.04,0.000000,0.040000,0.000000,0.040000,0.000000,0.040000,0.000000,0.000000,0.000000,0.080000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.04,0.000000,0.000000,0.000000,0.04,0.000000,0.04,0.000000,0.04,0.040000,0.04,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.08,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Little Portugal, Trinity",0.00,0.023256,0.00,0.046512,0.023256,0.000000,0.116279,0.023256,0.000000,0.000000,0.023256,0.000000,0.046512,0.00,0.000000,0.023256,0.069767,0.000000,0.000000,0.023256,0.023256,0.000000,0.023256,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.023256,0.00,0.000000,0.00,0.023256,0.000000,0.023256,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256,0.023256,0.023256,0.000000,0.000000,0.046512,0.00,0.000000,0.023256,0.000000,0.000000,0.023256,0.000000,0.023256,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.023256,0.046512,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.023256,0.0465

Let's print each neighborhood along with the top 5 most common venues

In [114]:
num_top_venues = 5
for hood in westtoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = westtoronto_grouped[westtoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.13
1  Performing Arts Venue  0.09
2         Breakfast Spot  0.09
3            Coffee Shop  0.09
4      Convenience Store  0.04


----Dufferin, Dovercourt Village----
                       venue  freq
0                   Pharmacy  0.11
1                     Bakery  0.11
2  Middle Eastern Restaurant  0.06
3              Grocery Store  0.06
4      Portuguese Restaurant  0.06


----High Park, The Junction South----
                venue  freq
0  Mexican Restaurant  0.08
1                Park  0.08
2     Thai Restaurant  0.08
3                Café  0.08
4           Speakeasy  0.04


----Little Portugal, Trinity----
                   venue  freq
0                    Bar  0.12
1            Coffee Shop  0.07
2            Men's Store  0.05
3       Asian Restaurant  0.05
4  Vietnamese Restaurant  0.05


----Parkdale, Roncesvalles----
            venue  freq
0  Breakfast Spot  0.14
1 

 let's write a function to sort the venues in descending order.

In [115]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [116]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = westtoronto_grouped['Neighborhood']

for ind in np.arange(westtoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(westtoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Nightclub,Convenience Store,Climbing Gym,Restaurant,Burrito Place,Italian Restaurant
1,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Café,Brewery,Music Venue,Park,Liquor Store,Pizza Place,Pool,Portuguese Restaurant
2,"High Park, The Junction South",Café,Thai Restaurant,Mexican Restaurant,Park,Antique Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,Italian Restaurant,Flea Market
3,"Little Portugal, Trinity",Bar,Coffee Shop,Restaurant,Men's Store,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Café,Korean Restaurant,Japanese Restaurant
4,"Parkdale, Roncesvalles",Gift Shop,Breakfast Spot,Cuban Restaurant,Bookstore,Dog Run,Eastern European Restaurant,Movie Theater,Coffee Shop,Restaurant,Dessert Shop


Run k-means to cluster the neighborhood into 5 clusters.

In [117]:
kclusters = 5

westtoronto_grouped_clustering = westtoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(westtoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 3, 2, 0, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [118]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

westtoronto_merged = westtoronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
westtoronto_merged = westtoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

westtoronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Café,Brewery,Music Venue,Park,Liquor Store,Pizza Place,Pool,Portuguese Restaurant
1,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,2,Bar,Coffee Shop,Restaurant,Men's Store,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Café,Korean Restaurant,Japanese Restaurant
2,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,4,Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Nightclub,Convenience Store,Climbing Gym,Restaurant,Burrito Place,Italian Restaurant
3,West Toronto,"High Park, The Junction South",43.661608,-79.464763,3,Café,Thai Restaurant,Mexican Restaurant,Park,Antique Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,Italian Restaurant,Flea Market
4,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Gift Shop,Breakfast Spot,Cuban Restaurant,Bookstore,Dog Run,Eastern European Restaurant,Movie Theater,Coffee Shop,Restaurant,Dessert Shop


Finally, let's visualize the resulting clusters

In [121]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(westtoronto_merged['Latitude'], westtoronto_merged['Longitude'], westtoronto_merged['Neighbourhood'], westtoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters